In [1]:
import os
import torch, detectron2
!nvcc --version
TORCH_VERSION = ".".join(torch.__version__.split(".")[:2])
CUDA_VERSION = torch.__version__.split("+")[-1]
print("torch: ", TORCH_VERSION, "; cuda: ", CUDA_VERSION)
print("detectron2:", detectron2.__version__)

zsh:1: command not found: nvcc
torch:  2.2 ; cuda:  2.2.1
detectron2: 0.6


In [2]:
# Some basic setup:
# Setup detectron2 logger
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

# import some common libraries
import numpy as np
import os, json, cv2, random
#from google.colab.patches import cv2_imshow
import matplotlib.pyplot as plt

# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog

from detectron2.data.datasets import register_coco_instances

In [3]:
dataset = 'cityscapes'
annot_dir = './cityscapes/annotations'
imgs_dir = './cityscapes'

for split in ['train', 'val']:
    #if split == 'train':
        #annot_path = '/kaggle/working/hope_to_public_net/instancesonly_filtered_gtFine_train_temp.json'
    #else:
    annot_path = os.path.join(annot_dir, f'instancesonly_filtered_gtFine_{split}.json')
    d_name = dataset + f'_{split}'
    register_coco_instances(d_name, {}, annot_path, imgs_dir)

In [4]:
dataset_name = 'cityscapes_train'

# Load dataset
dataset_dicts = DatasetCatalog.get(dataset_name)
metadata = MetadataCatalog.get(dataset_name)

[04/13 16:03:39 d2.data.datasets.coco]: Loading ./cityscapes/annotations/instancesonly_filtered_gtFine_train.json takes 2.25 seconds.
[04/13 16:03:39 d2.data.datasets.coco]: Loaded 2975 images in COCO format from ./cityscapes/annotations/instancesonly_filtered_gtFine_train.json


In [5]:
import argparse

class Args(argparse.Namespace):
    config_file='khang_net/configs/huflit_net/huflit_net_1x.py'
    eval_only=False
    num_gpus=1
    num_machines=1
    resume=True

args = Args()

In [6]:
from detectron2.model_zoo import get_config
from detectron2.config import LazyConfig
from detectron2.config.instantiate import instantiate

cfg = LazyConfig.load("khang_net/configs/huflit_net/huflit_net_1x.py")
cfg.train.device = 'mps'
cfg.dataloader.evaluator.dataset_name = 'cityscapes_val'
cfg.dataloader.train.dataset.names = 'cityscapes_train'
cfg.dataloader.test.dataset.names = 'cityscapes_val'
cfg.dataloader.train.total_batch_size = 16

"""
batch_on_paper = 16
actual_batch = cfg.dataloader.train.total_batch_size
lr_scale = actual_batch / batch_on_paper
cfg.optimizer.lr = cfg.optimizer.lr * lr_scale
cfg.optimizer.params.base_lr = cfg.optimizer.params.base_lr * lr_scale
cfg.optimizer.params.bias_lr_factor = cfg.optimizer.params.bias_lr_factor * lr_scale
cfg.optimizer.params.backbone_lr_factor = cfg.optimizer.params.backbone_lr_factor * lr_scale
"""

cfg.model.num_classes = 8
cfg.model.yolof.num_classes = 8
cfg.model.mask_head.num_classes = 8

#cfg.train.eval_period = 100000
#cfg.train.checkpointer.period = 1000
cfg.model.yolof_weight = None
cfg.model.train_yolof = True

cfg.optimizer.params.base_lr = 0.01
cfg.optimizer.lr = 0.01

cfg.train.max_iter = 30000
#cfg.train.init_checkpoint = '/Users/giakhang/Downloads/huflitnet_10k_iters/last_checkpoint'

In [7]:
from detectron2.engine import default_setup

default_setup(cfg, args)

[04/13 16:03:45 detectron2]: Rank of current process: 0. World size: 1
[04/13 16:03:45 detectron2]: Rank of current process: 0. World size: 1
[04/13 16:03:46 detectron2]: Environment info:
-------------------------------  -------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
sy

In [12]:
!cp /Users/giakhang/Downloads/huflit_net_20k_iters/last_checkpoint ./output
!cp /Users/giakhang/Downloads/huflit_net_20k_iters/model_final.pth ./output

In [14]:
from khang_net.lazyconfig_train_net import do_train

do_train(args, cfg)

[04/13 16:07:43 detectron2]: Model:
HUFLIT_Net(
  (yolof): YOLOF(
    (backbone): ResNet(
      (stem): BasicStem(
        (conv1): Conv2d(
          3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False
          (norm): FrozenBatchNorm2d(num_features=64, eps=1e-05)
        )
      )
      (res2): Sequential(
        (0): BottleneckBlock(
          (shortcut): Conv2d(
            64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False
            (norm): FrozenBatchNorm2d(num_features=256, eps=1e-05)
          )
          (conv1): Conv2d(
            64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False
            (norm): FrozenBatchNorm2d(num_features=64, eps=1e-05)
          )
          (conv2): Conv2d(
            64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False
            (norm): FrozenBatchNorm2d(num_features=64, eps=1e-05)
          )
          (conv3): Conv2d(
            64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False
            (no

RuntimeError: MPS backend out of memory (MPS allocated: 6.76 GB, other allocations: 2.09 GB, max allowed: 9.07 GB). Tried to allocate 220.50 MB on private pool. Use PYTORCH_MPS_HIGH_WATERMARK_RATIO=0.0 to disable upper limit for memory allocations (may cause system failure).

In [13]:
os.path.exists('./output/last_checkpoint')

True

In [8]:
!ls -al

total 25752
drwxr-xr-x@ 22 giakhang  staff      704 Apr 13 15:59 .
drwxr-xr-x@  6 giakhang  staff      192 Feb 29 11:03 ..
-rw-r--r--   1 giakhang  staff    10244 Apr 13 15:54 .DS_Store
drwxr-xr-x  15 giakhang  staff      480 Apr 12 12:17 .git
-rw-r--r--   1 giakhang  staff       32 Apr  4 22:32 .gitignore
drwxr-xr-x@  8 giakhang  staff      256 Apr  8 09:51 .ipynb_checkpoints
drwxr-xr-x@  3 giakhang  staff       96 Mar  8 16:39 .vscode
-rw-r--r--   1 giakhang  staff  2476527 Apr  4 22:32 HUFLIT_Net.ipynb
-rw-r--r--   1 giakhang  staff        0 Feb 29 11:18 README.md
-rw-r--r--@  1 giakhang  staff    79303 Apr 13 15:59 Resume training.ipynb
lrwxr-xr-x   1 giakhang  staff       66 Feb 29 11:16 cityscapes -> /Users/giakhang/dev/work/research_autopilot/dataset_zoo/cityscapes
drwxr-xr-x@ 27 giakhang  staff      864 Apr  4 22:32 detectron2
-rw-r--r--   1 giakhang  staff  2648564 Apr  4 22:32 inside_maskrcnn.ipynb
drwxr-xr-x@  8 giakhang  staff      256 Apr  5 10:00 khang_net
lrwxr-xr-x@  1 